### State Formulation
x for cross <br>
-- for empty <br>
o for circle <br>

In [ ]:
n=4
start=[['' for i in range(0,n)] for j in range(0,n)]
goal= [['' for i in range(0,n)] for j in range(0,n)]

In [ ]:
def match(state,des):
  for i in range(0,n):
    for j in range(0,n):
      if state[i][j]!=des[i][j]:
        return False
  return True

In [ ]:
import copy
def next_possible_move(state,goal, move):
  neighbours=[]
  for i in range(0,n):
    for j in range(0,n):
      if state[i][j]=='-':
        if goal[i][j]==move:
          next=copy.deepcopy(state)
          next[i][j]=move
          neighbours.append((next,(i,j),move))
        elif goal[i][j]=='-':
          continue
  return neighbours

In [ ]:
def valid_start_end(start,goal):
  x1,o1,e1=count(start)
  x2,o2,e2=count(goal)
  if x1>x2 or o1>o2:
    return False
  if (abs(x1-o1)==0 or abs(x1-o1)==1) and (abs(x2-o2)==0 or abs(x2-o2)==1):
    return True
  return False

### Helper

In [ ]:
def listtotuple(state):
  return tuple(map(tuple,state))
# listtotuple([['a','b','c'],['e','b','c']])

In [ ]:
def count(state):
  x=0
  o=0
  e=0
  for i in range(0,n):
    for j in range(0,n):
      if state[i][j]=='x':
        x+=1
      elif state[i][j]=='o':
        o+=1
      else:
        e+=1
  return x,o,e

In [ ]:
def reachable(state,goal):
  x1,o1,_=count(state)
  x2,o2,_=count(goal)
  if x1>=x2 or o1>=o2:
    return False
  return True

### Best First Search f(n)=h(n) [Greedy version]

#### Heurestic function

Properties

If all tiles match → h = 0 (best, goal state).

If you place in a cell that should be empty → big penalty (100).

If you mismatch → big penalty (100).

If you leave a goal cell empty → small penalty (1), guiding search to eventually fill it.

This works nicely with Greedy Best-First Search or A* because the min-heap pops the state with lowest heuristic first.

In [ ]:
def h(curr, goal):
    penalty_wrong_tile = 100   # large penalty for wrong symbol
    penalty_unwanted   = 100   # large penalty for filling empty goal cell
    reward_match       = 0     # no penalty if matched

    h = 0
    for i in range(3):
        for j in range(3):
            g = goal[i][j]
            c = curr[i][j]

            if g == '-' and c != '-':
                # placed something in a tile that should remain empty
                h += penalty_unwanted

            elif g != '-' and c == '-':
                # still empty but should be filled later
                h += 1   # small penalty = incomplete

            elif g != '-' and c != g:
                # mismatch (wrong symbol)
                h += penalty_wrong_tile

            # else: matched cell, no penalty

    return h


In [ ]:
import heapq

def gbfs(start, goal, move,depth):
    vis = set()

    # priority queue stores (heuristic, state, path_so_far, move)
    pq = [(h(start, goal), start, [], move)]

    while pq:
        hval, state, path, move = heapq.heappop(pq)
        #print(state)
        if depth ==0:
          return path + [(state, hval, (-1, -1), 'Depth Reached')]

        depth=depth-1

        if listtotuple(state) in vis:
            continue
        vis.add(listtotuple(state))

        if match(state, goal):
            return path + [(state, hval, (-1, -1), 'Goal Reached')]

        for neighbour, act, move in next_possible_move(state, goal,move):
            move1 = 'o' if move == 'x' else 'x'
            if listtotuple(neighbour) not in vis:
                heapq.heappush(
                    pq,
                    (
                        h(neighbour, goal),  # <-- only h(n)
                        neighbour,
                        path + [(state, hval, act, move)],
                        move1
                    )
                )

    if not reachable(state, goal):
        return path + [(state, 0, (-1, -1), 'Not reachable')]

In [ ]:
print(gbfs(start,goal,'x',50))

[([['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', '']], 0, (-1, -1), 'Goal Reached')]


### Input File

In [ ]:
def input(filename):
  with open(filename,"r") as f:
    lines = f.readlines()

  state=[['' for i in range(0,n)] for j in range(0,n)]
  for i in range(0,n):
    line=lines[i].strip()
    part=line.split()
    # Check if the number of elements in 'part' matches 'n'
    if len(part) != n:
        raise ValueError(f"Incorrect number of elements in line {i+1} of the input file.")
    for j in range(0,n):
      state[i][j]=part[j]
  start=state

  state=[['' for i in range(0,n)] for j in range(0,n)]
  for i in range(0,n):
    line=lines[i+n].strip()
    part=line.split()
    # Check if the number of elements in 'part' matches 'n'
    if len(part) != n:
        raise ValueError(f"Incorrect number of elements in line {i+1+3} of the input file.")
    for j in range(0,n):
      state[i][j]=part[j]
  goal=state

  print(start)
  print(goal)
  # print(start==goal)
  if valid_start_end(start,goal)==False:
    soln=[(start,0,(-1,-1),'Invalid Start State to reach end')]
  else:
    x,o,_=count(start)
    if x>o:
      soln=(gbfs(start,goal,'o',500))
    else:
      soln=(gbfs(start,goal,'x',500))

  if soln is None:
    print("Goal not reached within the given depth.")
    # Optionally, you could write a message to the output file as well
    with open("output.txt", "w") as f:
      f.write("Goal not reached within the given depth.\n")
  else:
    print(soln)
    output("output.txt",soln)

In [ ]:
input("input.txt")

[['x', '-', '-', '-'], ['-', '-', '-', '-'], ['-', '-', '-', '-'], ['-', '-', '-', '-']]
[['x', 'o', 'o', 'x'], ['x', '-', '-', '-'], ['-', '-', '-', '-'], ['-', '-', '-', '-']]
[([['x', '-', '-', '-'], ['-', '-', '-', '-'], ['-', '-', '-', '-'], ['-', '-', '-', '-']], 3, (0, 2), 'o'), ([['x', '-', 'o', '-'], ['-', '-', '-', '-'], ['-', '-', '-', '-'], ['-', '-', '-', '-']], 2, (1, 0), 'x'), ([['x', '-', 'o', '-'], ['x', '-', '-', '-'], ['-', '-', '-', '-'], ['-', '-', '-', '-']], 1, (0, 1), 'o'), ([['x', 'o', 'o', '-'], ['x', '-', '-', '-'], ['-', '-', '-', '-'], ['-', '-', '-', '-']], 0, (0, 3), 'x'), ([['x', 'o', 'o', 'x'], ['x', '-', '-', '-'], ['-', '-', '-', '-'], ['-', '-', '-', '-']], 0, (-1, -1), 'Goal Reached')]


In [ ]:
def output(filename,soln):
  with open(filename,"w") as f:
    for line in soln:
      state,cost,act,move=line
      for row in state:
        f.write(" ".join(row))
        f.write("\n")
      f.write("\n")
      if act!=(-1,-1):
        f.write(f"Place {move} in position {str(act)}")
        f.write("\n")
      else:
        f.write(str(move))
        f.write("\n")